# **Amazon Sales Data**

For this project, I used the Amazon Sales data from kaggle: https://www.kaggle.com/datasets/arpit2712/amazonsalesreport

**In this project, I will be answering the following questions:**

1. What is the total revenue generated?
2. What is the total Quantity Ordered?
3. What is the month wise total revenue generated and quantity ordered?
4. What is the most popular product category in terms of sales?
5. What are the three top state in terms of sales revenue?
6. What is the overall percentage of Order status?

# Step Zero: Data Exploration

To begin the project, Let's take a quick glance at the dataset

In [ ]:
SELECT TOP 100 * 
FROM Amazon_sales


This is the great, complete dataset containing information such as the Order id, order date, category, status, size, courier status, quantity, Amount, ship state.Lots of things to analyse here.

# Step One: Data Cleaning

I added the total amount in case of a purchase with more than one quantity. The text formatting seems to be consistent across columns, so there is no need to LOWER or UPPER each one.

In [ ]:
ALTER TABLE amazon_sales
ADD Total_Amount FLOAT

UPDATE Amazon_sales SET Total_Amount = Amount*Qty 

Checking for the Duplicates in Order id column. There must be one order id per order. If there are more than one order id then those are duplicate rows.

In [ ]:
SELECT Order_ID, 
    count(*) AS c
FROM Amazon_sales
GROUP BY Order_ID
HAVING  count(*) >1;

Output: 6988. 
There are 6988 duplicate rows.

Removing the duplicate rows by deleting them.

In [ ]:
WITH CTE AS (
    SELECT 
        Order_ID, 
        ROW_NUMBER() OVER (PARTITION BY Order_ID ORDER BY (SELECT NULL)) AS rn
    FROM 
        Amazon_sales
)
DELETE 
FROM CTE 
WHERE rn > 1;


Next, Let's check for Null values in the Total Amount column as this was a custom computed column which we will use for later analysis.

In [ ]:
SELECT *
FROM Amazon_sales
WHERE total_amount IS NULL;

Output: 0. Excellent, there are no Null values in this column.

Now,we will modify the Amazon sales table by adding a new column Order_Status and populating it based on the current Status and courier_status fields. The Order_Status is categorized into specific values: 'Successful_Delivery', 'Delivery_issues', 'Order_Processing', 'Shipping_in_progress', and 'Order_Cancelled', depending on the conditions in the existing data, thereby enabling easier tracking and analysis of order progress and issues.

In [ ]:
ALTER TABLE Amazon_sales
ADD Order_Status varchar(max);

UPDATE Amazon_sales SET Order_Status =	(CASE WHEN Status IN ('Pending','Pending - Waiting for Pick Up') OR courier_status ='Unshipped' THEN 'Order_Processing'
              WHEN Status IN ('Shipping','Shipped','Shipped - Out for Delivery','Shipped - Picked Up') OR courier_status IN ('Shipped','On the Way') THEN 'Shipping_in_progress'
              WHEN Status IN ('Shipped - Delivered to Buyer') THEN 'Successful_Delivery'
              WHEN Status IN ('Shipped - Returned to Seller','Shipped - Returning to Seller','Shipped - Damaged','Shipped - Rejected by Buyer','Shipped - Lost in Transit')  THEN 'Delivery_issues'
              ELSE 'Order_Cancelled' END ) FROM Amazon_sales

# Step Two: Analyse the data

**Q1: What is the total revenue generated?**

In [ ]:
SELECT ROUND(SUM(Total_amount),2) AS Total_revenue
FROM Amazon_sales;

Output: 712,10,786.07 rupees

**Q2: What is the total Quantity Ordered?**

In [ ]:
SELECT SUM(qty) AS total_quantity
FROM Amazon_sales;

Output:1,02,260 units

**Q3: What is the month wise total revenue generated and quantity ordered?**

In [ ]:
SELECT  DATENAME(MONTH,order_Date) AS MONTH,
        ROUND(SUM(total_amount),2) AS revenue,
         SUM(qty) AS Total_quantity
FROM Amazon_sales
GROUP BY 
        DATENAME(MONTH,order_Date),
        MONTH(order_date)
ORDER BY 
        MONTH(order_date);

 We have four months i.e.,march,april,may,june's data. The revenue generated and Quantity ordered in these months are following:

Output: 
1.        Month  : Revenue     : Total_quantity
1.      March  : 91965.79    : 134
1.      April  : 26066020.46 : 38693
1.      May    : 23667569.34 : 33392
1.      June   : 21385230.49 : 30041

**Q4: What is the most popular product category in terms of sales?**

In [ ]:
SELECT 
    category,
    ROUND(SUM(qty),2) AS total_Quantity
FROM 
    Amazon_sales
GROUP BY
    Category
ORDER BY
    total_Quantity DESC; 

Output: T-Shirt is the most popular product in terms of sales.

1. T-shirt: 40,155 units
1. Shirt: 38,486 units
1. Blazzer: 12,599 units
1. Trousers: 8,847 units
1. Perfume: 910 units
1. Wallet: 802 units
1. Socks: 344 units
1. Shoes: 117 units

**Q5: What are the three top state in terms of sales revenue?**

In [ ]:
SELECT TOP 3
    ship_state,
    ROUND(SUM(total_amount),2) AS Total_revenue
FROM 
    Amazon_sales
GROUP BY 
    ship_state
ORDER BY
    Total_revenue DESC;

Output: The top three highest sales revenue state are the following:

1. Maharastra: 12032453.09
1. Karnataka: 9565952.48
1. Uttar Pradesh: 6280864.27

**Q6: What is the overall percentage of Order status?**

In [ ]:
SELECT 
       order_status,
       ROUND(COUNT(order_status)*100.0/(SELECT COUNT(*) FROM Amazon_sales),2) AS percentage
FROM 
        Amazon_sales
GROUP BY 
        order_status
ORDER BY 
        percentage desc;

Output: Percent of order status 

1. Shipping_in_progress: 66.62
1. Successful_Delivery: 21.84
1. Order_Processing: 5.13
1. Order_Cancelled: 4.73
1. Delivery_issues: 1.67

# Step Three: Conclusions

Based on the comprehensive analysis of the Amazon sales data of four months:

1. The Total revenue generated was  712,10,786.07 rupees.

1. The Quantity ordered was 1,02,260 units.

1. The monthly generated revenue and the quantity ordered are as follows:
    *     Month : Revenue : Total_quantity
    *     March : 91965.79 : 134
    *     April : 26066020.46 : 38693
    *     May : 23667569.34 : 33392
    *     June : 21385230.49 : 30041
   

1. The most popular product in terms of sales was 'T-shirt' with 40,155 units sold, significantly outperforming other products.

1. The top three states with the highest sales revenue were: 
    * Maharastra: 120,32,453.09 rupees
    * Karnataka: 95,65,952.48 rupees
    * Uttar Pradesh: 62,80,864.27 rupees
    

1. The overall percentage of the order status were:
    * Shipping_in_progress: 66.62%
    * Successful_Delivery: 21.84%
    * Order_Processing: 5.13%
    * Order_Cancelled: 4.73%
    * Delivery_issues: 1.67%
    
 
 
*In essence, the data reflects a strong preference for 'T-shirt' across shoppers and leading purchases are from 'Maharastra' which generates more revenue compare to other states.*
 
 
 





**Thankyou for reading this SQL project , I appreciate it!**